# Module Importing

In [1]:
import os
os.chdir('..')

In [2]:
import datetime

In [3]:
import pandas as pd
import numpy as np

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
%matplotlib inline

# Data Loading

## Data fields
+ **ID** - an Id that represents a (Shop, Item) tuple within the test set
+ **shop_id** - unique identifier of a shop
+ **item_id** - unique identifier of a product
+ **item_category_id** - unique identifier of item category
+ **item_cnt_day** - number of products sold. You are predicting a monthly amount of this measure
+ **item_price** - current price of an item
+ **date** - date in format dd/mm/yyyy
+ **date_block_num** - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
+ **item_name** - name of item
+ **shop_name** - name of shop
+ **item_category_name** - name of item category

## Sales_train
**(the training set. Daily historical data from January 2013 to October 2015)**

In [7]:
sales_train_df = pd.read_csv('data/sales_train.csv')

In [8]:
sales_train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [9]:
print('Shape of sales_train.csv is (%d, %d)' % sales_train_df.shape)

Shape of sales_train.csv is (2935849, 6)


## Items
**(supplemental information about the items/products)**

In [10]:
items_df = pd.read_csv('data/items.csv')

In [11]:
items_df.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [12]:
print('Shape of items.csv is (%d, %d)' % items_df.shape)

Shape of items.csv is (22170, 3)


## Item_categories
**(supplemental information about the items categories)**

In [13]:
item_categories_df = pd.read_csv('data/item_categories.csv')

In [14]:
item_categories_df.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [15]:
print('Shape of item_categories.csv is (%d, %d)' % item_categories_df.shape)

Shape of item_categories.csv is (84, 2)


## Shops
**(supplemental information about the shops)**

In [16]:
shops_df = pd.read_csv('data/shops.csv')

In [17]:
shops_df.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [18]:
print('Shape of shops.csv is (%d, %d)' % shops_df.shape)

Shape of shops.csv is (60, 2)


## test
**(the test set. You need to forecast the sales for these shops and products for November 2015)**

In [19]:
test_df = pd.read_csv('data/test.csv')

In [20]:
test_df.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [21]:
print('Shape of test.csv is (%d, %d)' % test_df.shape)

Shape of test.csv is (214200, 3)


# Basic EDA

# Basic data preparation

### 1.  Adding revenue column

In [22]:
# sales_train_df['revenue'] = sales_train_df['item_price'] * sales_train_df['item_cnt_day']

In [23]:
# sales_train_df.head()

### 2.  Date column to correct format

In [24]:
sales_train_df['date'] = sales_train_df['date'].apply(lambda x: datetime.datetime.strptime(x, '%d.%m.%Y'))

### 3.  Adding information about category

In [25]:
sales_train_df = sales_train_df.join(items_df, on='item_id', rsuffix='_').drop(['item_id_', 'item_name'], axis=1)

### 4.  Droping duplicates

In [26]:
subset = ['date','date_block_num','shop_id','item_id','item_cnt_day']

In [27]:
print('Duplicates: {}'.format(sales_train_df.duplicated(subset=subset).sum()))

Duplicates: 24


In [28]:
sales_train_df.drop_duplicates(subset=subset, inplace=True)

### 5.  Dropping shops and items not in test data

In [29]:
test_shops = test_df.shop_id.unique()
test_items = test_df.item_id.unique()

In [30]:
train = sales_train_df[sales_train_df.shop_id.isin(test_shops)]
train = sales_train_df[sales_train_df.item_id.isin(test_items)]

### 6.  Creating all combinations

In [31]:
from itertools import product

In [32]:
block_shop_combi = pd.DataFrame(list(product(np.arange(34), test_shops)), columns=['date_block_num','shop_id'])

In [33]:
shop_item_combi = pd.DataFrame(list(product(test_shops, test_items)), columns=['shop_id','item_id'])

In [34]:
all_combi = pd.merge(block_shop_combi, shop_item_combi, on=['shop_id'], how='inner')
print('all_combi length = {}'.format(len(all_combi)))

all_combi length = 7282800


In [35]:
train = pd.merge(all_combi, sales_train_df, on=['date_block_num','shop_id','item_id'], how='left')

In [36]:
train['item_cnt_day'].fillna(0, inplace=True)

### 7.  Creating month sales table

In [37]:
train_month = train.groupby(['date_block_num','shop_id','item_id']) \
                   .agg({'item_cnt_day': ['sum','count']}) \
                   .reset_index()

In [38]:
train_month.columns = ['date_block_num','shop_id',
                       'item_id','item_cnt','item_order']

In [39]:
print(train_month[['item_cnt','item_order']].describe())

           item_cnt    item_order
count  7.282800e+06  7.282800e+06
mean   2.221345e-01  1.085718e+00
std    3.324564e+00  7.254517e-01
min   -4.000000e+00  1.000000e+00
25%    0.000000e+00  1.000000e+00
50%    0.000000e+00  1.000000e+00
75%    0.000000e+00  1.000000e+00
max    2.253000e+03  3.100000e+01


In [40]:
train_month['item_cnt'].clip(0, 20, inplace=True)

In [41]:
train_month.head()

,date_block_num,shop_id,item_id,item_cnt,item_order
0,0,2,30,0.0,1
1,0,2,31,0.0,1
2,0,2,32,0.0,1
3,0,2,33,1.0,1
4,0,2,38,0.0,1


In [58]:
train_month.T

,0,1,2,3,4,5,6,7,8,9,...,7282790,7282791,7282792,7282793,7282794,7282795,7282796,7282797,7282798,7282799
date_block_num,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0
shop_id,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,59.0,59.0,59.0,59.0,59.0,59.0,59.0,59.0,59.0,59.0
item_id,30.0,31.0,32.0,33.0,38.0,42.0,45.0,51.0,53.0,57.0,...,22118.0,22137.0,22139.0,22145.0,22154.0,22162.0,22163.0,22164.0,22166.0,22167.0
item_cnt,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
item_order,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### 8.  Feature creation

In [45]:
item_group = item_categories_df['item_category_name'].apply(lambda x: str(x).split(' ')[0])

In [46]:
item_categories_df['item_group'] = pd.Categorical(item_group).codes

In [48]:
items_df = pd.merge(items_df,
                    item_categories_df.loc[:,['item_category_id','item_group']],
                    on=['item_category_id'], how='left')

In [50]:
item_group.unique()

array(['PC', 'Аксессуары', 'Билеты', 'Доставка', 'Игровые', 'Игры',
       'Карты', 'Кино', 'Книги', 'Музыка', 'Подарки', 'Программы',
       'Служебные', 'Чистые', 'Элементы'], dtype=object)

In [52]:
city = shops_df.shop_name.apply(lambda x: str.replace(x, '!', '')) \
                         .apply(lambda x: x.split(' ')[0])

In [54]:
shops_df['city'] = pd.Categorical(city).codes

In [55]:
city.unique()

array(['Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж',
       'Выездная', 'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга',
       'Коломна', 'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород',
       'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев',
       'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Цифровой', 'Чехов',
       'Ярославль'], dtype=object)